In [20]:
import pandas as pd
import re
from newspaper import Article

In [2]:
link = 'https://www.nytimes.com/2017/09/01/opinion/artificial-intelligence-regulations-rules.html'

In [3]:
article = Article(link)
article.download()
article.parse()
article.title

'How to Regulate Artificial Intelligence'

In [5]:
import requests
from bs4 import BeautifulSoup

In [18]:
links = []
soup = BeautifulSoup(open("nytimes-cryptocurrency-1.html"), "html.parser")

In [28]:
for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
    links.append(link.get('href'))
 
print(links)

['https://www.nytimes.com/', 'https://query.nytimes.com/', 'https://www.nytimes.com/reuters/2017/12/14/technology/14reuters-siebertfinancial-stocks.html', 'https://www.nytimes.com/2017/08/03/style/what-is-cryptocurrency.html', 'https://www.nytimes.com/2017/08/03/style/what-is-cryptocurrency.html', 'https://www.nytimes.com/reuters/2017/12/21/business/21reuters-russia-bitcoin-law-draft.html', 'https://www.nytimes.com/reuters/2017/12/19/business/19reuters-the-crypto-company-suspension.html', 'https://www.nytimes.com/reuters/2017/12/22/business/22reuters-cryptocurrency-companies.html', 'https://www.nytimes.com/reuters/2017/12/22/business/22reuters-cryptocurrency-companies-factbox.html', 'https://www.nytimes.com/reuters/2017/12/21/business/21reuters-global-bitcoin.html', 'https://www.nytimes.com/aponline/2017/12/21/business/ap-us-long-island-iced-tea-name-change.html', 'https://www.nytimes.com/reuters/2017/12/19/business/19reuters-coinbase-bitcoin-cash.html', 'https://www.nytimes.com/reuter

In [32]:
urls = []
for num in range(1, 10):
    file = "nytimes-cryptocurrency-" + str(num) + ".html"
    soup = BeautifulSoup(open(file), "html.parser")
    
    for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
        urls.append(link.get('href'))

In [34]:
import csv

with open('nytimes-cryptocurrency-urls.csv', 'w') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(urls)
    